In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Load weather data and create data fram
weather_data = "output_data/weather_data.csv"

city_df = pd.read_csv(weather_data)
city_df.head()

,city,temp,country,date,latitude,longitude,cloudiness,humidity,wind speed
0,Chokurdakh,-47.76,RU,1639071633,70.6333,147.9167,87,100,2.73
1,Saint-Philippe,78.39,RE,1639071725,-21.3585,55.7679,55,78,12.33
2,Saldanha,69.85,ZA,1639071475,-33.0117,17.9442,81,91,23.40
3,Fortuna,48.36,US,1639071725,40.5982,-124.1573,1,80,1.99
4,Kaitangata,55.92,NZ,1639071725,-46.2817,169.8464,94,81,3.71


In [3]:
#Configure gmaps and Use the Lat and Lng as locations and Humidity as the weight
gmaps.configure(api_key=g_key)
locations = city_df[["latitude", "longitude"]]
humidity = city_df["humidity"]
temp=city_df["temp"]

#Add Heatmap layer to map.
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create new DataFrame fitting weather criteria
#Ideal weather conditions
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph and Zero cloudiness.

weather_df = city_df.loc[(city_df["temp"] >=70) & (city_df["temp"] <=80) & (city_df["wind speed"] <10) & (city_df["cloudiness"] == 0)]

hotel_df = weather_df.dropna(how='any')
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,city,temp,country,date,latitude,longitude,cloudiness,humidity,wind speed
0,Castro,75.65,BR,1639071139,-24.7911,-50.0119,0,33,7.02
1,Ouadda,77.05,CF,1639071145,8.0777,22.4007,0,24,5.68
2,East London,72.54,ZA,1639071146,-33.0153,27.9116,0,88,4.61
3,Adré,73.76,TD,1639071190,13.4667,22.2000,0,16,8.59


In [5]:
#Add hotel name column to the dataframe
hotel_df["hotel name"] = ""
hotel_df

,city,temp,country,date,latitude,longitude,cloudiness,humidity,wind speed,hotel name
0,Castro,75.65,BR,1639071139,-24.7911,-50.0119,0,33,7.02,
1,Ouadda,77.05,CF,1639071145,8.0777,22.4007,0,24,5.68,
2,East London,72.54,ZA,1639071146,-33.0153,27.9116,0,88,4.61,
3,Adré,73.76,TD,1639071190,13.4667,22.2000,0,16,8.59,


In [6]:
#Set parameters to search for hotels with 5000 meters
numhotel = 0

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    if numhotel == 10:
        break
    
    lat = row["latitude"]
    lon = row["longitude"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        numhotel = numhotel + 1
        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = None

In [7]:
hotel_df

,city,temp,country,date,latitude,longitude,cloudiness,humidity,wind speed,hotel name,Hotel Name
0,Castro,75.65,BR,1639071139,-24.7911,-50.0119,0,33,7.02,,CHACARA BAILLY
1,Ouadda,77.05,CF,1639071145,8.0777,22.4007,0,24,5.68,,None
2,East London,72.54,ZA,1639071146,-33.0153,27.9116,0,88,4.61,,Tu Casa
3,Adré,73.76,TD,1639071190,13.4667,22.2000,0,16,8.59,,None


In [8]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))